In [27]:
def loadGraph(filename):
    with open(str(filename), "r") as graph_instance:
        chromatic_num = int(graph_instance.readline())
        V, E = list(map(int, graph_instance.readline().split(" ")))
        graph = [[] for _ in range(V)]
        for line in graph_instance:
            if line.startswith('e'):
                line = line.split(" ")
                del line[0]
                edge = list(map(int, line))
                if (edge[1]-1) not in graph[edge[0]-1]:
                    graph[edge[0] - 1].append(edge[1] - 1)
                    graph[edge[1] - 1].append(edge[0] - 1)
    return chromatic_num, graph

In [28]:
def printSolution(color):
    print("Solution: ", end = " ")
    for i in range(len(color)):
        print(str(color[i]), end=" ")
        
    print()

In [29]:
import random
import numpy as np

In [30]:
class Individual:
    
    def __init__(self, graph):
        self.graph = graph
        self.V = len(graph)
        self.code = []
        self.getCode()
        self.fitness = self.calcFitness()
        
    def __str__(self):
        return str(self.code)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def isFeasible(self):
        for i in range(len(self.graph)):
            for neigh in self.graph[i]:
                if(self.code[i]==self.code[neigh]):
                    return False
            
        return True
    
    def randomChangeOperator(self):
        position = random.randrange(len(self.graph))
        old_color = self.code[position]
      
        new_color = random.randrange(1, len(self.graph)+1)
        
        self.code[position] = new_color
        if self.isFeasible():
            return position, old_color
        
        self.code[position] = old_color
        return -1, old_color
    
    def getCode(self):
        for i in range(self.V):
            self.code.append(random.randrange(1, self.V+1))
        
    def getAvailableColors(self, i):
        neighbors = [index for index in self.graph[i]]
        used_colors = [self.code[neighbor] for neighbor in neighbors]
        available_colors = [c for c in range(1, self.V+1) if c not in used_colors]
        return available_colors
    
    def repair(self):
        for i in range(self.V):
            neighbors = [index for index in self.graph[i]]
            used_colors = [self.code[neighbor] for neighbor in neighbors]
            if(self.code[i] in used_colors):  
                available_colors = [c for c in range(1, self.V+1) if c not in used_colors]
                self.code[i] = random.choice(available_colors)
        
    def calcFitness(self):
        penalty = 0
        for i in range(self.V):
            for j in self.graph[i]:
                if self.code[i] == self.code[j]:
                    penalty += 1
        return penalty + len(set(self.code))

In [31]:
def selection(population):
    TOURNAMENT_SIZE = 5
    min_fitness = float('inf')
    index = -1
    
    for i in range(TOURNAMENT_SIZE):
        candidate = random.randrange(len(population))
        if population[candidate].fitness < min_fitness:
            min_fitness = population[candidate].fitness
            index = candidate
    return index 

In [32]:
def crossover(parent1, parent2, child1, child2):
    position = random.randrange(1, parent1.V-1)
    child1.code[:position] = parent1.code[:position]
    child2.code[:position] = parent2.code[:position]
    
    child1.code[position:] = parent2.code[position:]
    child2.code[position:] = parent1.code[position:]

In [33]:
def mutation(individual):
    MUTATION_PROB = 0.8
    p = random.random()
    if p < MUTATION_PROB:
        i = random.randrange(individual.V)
        old_color = individual.code[i]
        individual.code[i] = random.choice(individual.getAvailableColors(i))
        
        if individual.code[i] == old_color:
            individual.code[i] = random.randrange(1, individual.V)
    
    return individual

In [34]:
 def simulatedAnnealing(individual, chromatic_num, max_iters):
    for i in range(1, max_iters+1):
        if len(set(individual.code)) == chromatic_num:
            break
        
        position, old_color = individual.randomChangeOperator()
      
        if position < 0:
            continue
        
        new_value = individual.calcFitness()
        
        if new_value < individual.fitness:
            individual.fitness = new_value
        else:
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = new_value
            else:
                individual.code[position] = old_color

In [35]:
POPULATION_SIZE = 50
ELITISIM_SIZE = 4

In [36]:
def GeneticAlgorithm(graph, chromatic_num, NUM_GENERATIONS):
    population = [Individual(graph) for _ in range(POPULATION_SIZE)]
    newPopulation = [Individual(graph) for _ in range(POPULATION_SIZE)]

    
    for i in range(NUM_GENERATIONS):        
        population.sort()
        newPopulation[:ELITISIM_SIZE] = population[:ELITISIM_SIZE]
    
        for j in range(ELITISIM_SIZE, POPULATION_SIZE, 2):
            parent1Index = selection(population)
            parent2Index = selection(population)
        
            crossover(population[parent1Index], population[parent2Index], newPopulation[j], newPopulation[j+1])
        
            mutation(newPopulation[j])
            mutation(newPopulation[j+1])
        
            newPopulation[j].fitness = newPopulation[j].calcFitness()
            newPopulation[j+1].fitness = newPopulation[j+1].calcFitness()
        simulatedAnnealing(min(newPopulation), chromatic_num, 10)
        population=newPopulation
    
        bestIndividual = min(population)
        bestValue = len(set(bestIndividual.code))
    
        if(bestValue == chromatic_num):
            break
    
    population.sort()

    bestIndividual = min(population)
    num_of_colors = len(set(bestIndividual.code))
 
    return num_of_colors, bestIndividual.code, i+1

In [39]:
graph_list = ['brute.txt', 'myciel3.col', 'myciel4.col', 'myciel5.col', 'myciel6.col', 'david.col', 'huck.col', 'jean.col']
import time
from termcolor import colored

for graph_file in graph_list:
            print()
            print(colored(graph_file, color="red"))
            for max_generations in [100, 1000, 10000]:
                graph = []
                chromatic_num, graph = loadGraph(graph_file)
                
                print(f'---------------------Max generations: {max_generations}---------------------')
                start = time.time()
                num_of_colors, color, num_of_iters = GeneticAlgorithm(graph, chromatic_num, max_generations)
                end = time.time()
                exec_time = end - start
                
                print('time: ', exec_time)
                printSolution(color)
                print(f'Chromatic number: {chromatic_num}, Found chromatic number: {num_of_colors}')
                print(f'Number of iters: {num_of_iters}')


brute.txt
---------------------Max generations: 100---------------------
time:  0.0016734600067138672
Solution:  5 3 1 2 1 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 1
---------------------Max generations: 1000---------------------
time:  0.00235748291015625
Solution:  2 1 5 3 5 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 1
---------------------Max generations: 10000---------------------
time:  0.003928422927856445
Solution:  3 4 1 2 1 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 1

myciel3.col
---------------------Max generations: 100---------------------
time:  0.010028600692749023
Solution:  5 7 10 10 7 3 3 10 3 7 5 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 7
---------------------Max generations: 1000---------------------
time:  0.00815129280090332
Solution:  5 2 10 2 9 9 9 5 2 5 10 
Chromatic number: 4, Found chromatic number: 4
Number of iters: 7
---------------------Max generations: 10000------